# init

In [1]:
import comet_ml
import datatable as dt
import pandas as pd
import pyarrow.feather as feather
import pyarrow as pa

from tqdm.auto import tqdm
from datatable import f, update

dt.init_styles()

# Download Comet

## Download `y_car`

In [4]:
%%time
comet_api = comet_ml.API()
exps = comet_api.query('amiao', 'earnings-call', 
                       (comet_ml.api.Parameter('note').startswith('STL-49') & \
                       (comet_ml.api.Metric('test_rmse')!=None)),
                       archived=False)

y_car = dt.Frame()
for exp in exps:
    note = exp.get_parameters_summary('note')['valueCurrent']
    model_name = note[:6]
    try:
        asset_id = [asset['assetId'] for asset in exp.get_asset_list() 
                    if asset['fileName']=='y_car.feather'][0]
        roll_type = exp.get_parameters_summary('roll_type')['valueCurrent']
        yqtr = exp.get_parameters_summary('yqtr')['valueCurrent']

        y = dt.Frame(feather.read_feather(pa.BufferReader(exp.get_asset(asset_id))))
    except Exception as e:
        print(e)
        print(exp.get_name())
        print(asset_id)
        print('-----------')
    y[:, update(yqtr=yqtr, model_name=model_name, note=note, roll_type=roll_type)]
    
    y_car.rbind(y, force=True, bynames=True)

# append new results to existing comet log
old_y_car = feather.read_feather('data/y_car_comet.feather')
all_y_car = pd.concat([old_y_car, y_car.to_pandas()], ignore_index=True)
all_y_car = pd.DataFrame.drop_duplicates(all_y_car, ignore_index=True)
all_y_car = all_y_car[all_y_car.note.notnull()]

feather.write_feather(all_y_car, 'data/y_car_comet.feather', compression='uncompressed');

Wall time: 12 s


## Download parameters

In [5]:
%%time

comet_api = comet_ml.API()
exps = comet_api.query('amiao', 'earnings-call', 
                       (comet_ml.api.Parameter('note').startswith('STL-49') & \
                       (comet_ml.api.Metric('test_rmse')!=None)),
                       archived=False)

params_comet = []
for exp in exps:
    # get parameter
    log = {param['name']:param['valueCurrent'] for param in exp.get_parameters_summary()}
    
    # get metadat
    log = {**log, **exp.get_metadata()}
    
    # delete useless params
    for key in ['checkpoint_path', 'f']:
        log.pop("key", None)
    params_comet.append(log)
    

params_comet = pd.DataFrame(params_comet)

# append new results to existing comet log
old_params_comet = feather.read_feather('data/params_comet.feather')
all_params_comet = pd.concat([old_params_comet, params_comet], ignore_index=True)
all_params_comet = pd.DataFrame.drop_duplicates(all_params_comet, ignore_index=True);
feather.write_feather(all_params_comet, 'data/params_comet.feather', compression='uncompressed');

Wall time: 8.39 s


# Modify param

In [40]:
query = ((comet_ml.api.Parameter('roll_type')=='5y') &
         (comet_ml.api.Parameter('note')=='STL-39,bsz=32,lr=3.00e-04'))

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in exps:
    # roll_type = exp.get_parameters_summary('roll_type').valueCurrent
    exp.log_parameter('note', 'STL-40,bsz=32,lr=3.00e-04')

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

...

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

# upload `test_start`

In [ ]:
split_df = pd.read_csv(f'data/split_dates.csv')

query = ((comet_ml.api.Parameter('test_start')==None) & \
         (comet_ml.api.Parameter('test_end')==None) & \
         (comet_ml.api.Parameter('train_start')==None) & \
         (comet_ml.api.Parameter('train_end')==None))

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in tqdm(exps):
    window = exp.get_parameters_summary('window')['valueCurrent']
    roll_type = exp.get_parameters_summary('roll_type')['valueCurrent']
         
    train_start = split_df.loc[(split_df.roll_type==roll_type) & (split_df.window==window)].train_start.to_list()[0]
    train_end = split_df.loc[(split_df.roll_type==roll_type) & (split_df.window==window)].train_end.to_list()[0]
    test_start = split_df.loc[(split_df.roll_type==roll_type) & (split_df.window==window)].test_start.to_list()[0]         
    test_end = split_df.loc[(split_df.roll_type==roll_type) & (split_df.window==window)].test_end.to_list()[0]
         
    # print(f'{roll_type}-{test_start}')
         
    exp.log_parameter('train_start', train_start)
    exp.log_parameter('train_end', train_end)
    exp.log_parameter('test_start', test_start)
    exp.log_parameter('test_end', test_end)

# upload `OLS_rmse` 

In [19]:
bench_fr = pd.read_feather('data/bench_fr.feather')

query = (comet_ml.api.Parameter('ols_rmse_norm')!=None)

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in tqdm(exps):
    test_start = exp.get_parameters_summary('test_start')['valueCurrent'] # char
    current_ols_rmse = float(exp.get_parameters_summary('ols_rmse_norm')['valueCurrent'])
    
    true_ols_rmse = bench_fr.loc[(bench_fr.roll_type=='3y') & (bench_fr.test_start==pd.to_datetime(test_start))].test_rmse_fr_norm.to_list()[0]
    
    if current_ols_rmse != true_ols_rmse:
        exp.log_parameter('ols_rmse_norm', true_ols_rmse)

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

...

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}